In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.layers import common_attention
from tensor2tensor.utils import registry
from tensor2tensor import problems
import tensorflow as tf
import os
import logging
import sentencepiece as spm
import transformer_tag
from tensor2tensor.layers import modalities

In [3]:
vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100

    def encode(self, s):
        return self.sp.EncodeAsIds(s)

    def decode(self, ids, strip_extraneous = False):
        return self.sp.DecodeIds(list(ids))

In [4]:
d = [
    {'class': 0, 'Description': 'PAD', 'salah': '', 'betul': ''},
    {
        'class': 1,
        'Description': 'kesambungan subwords',
        'salah': '',
        'betul': '',
    },
    {'class': 2, 'Description': 'tiada kesalahan', 'salah': '', 'betul': ''},
    {
        'class': 3,
        'Description': 'kesalahan frasa nama, Perkara yang diterangkan mesti mendahului "penerang"',
        'salah': 'Cili sos',
        'betul': 'sos cili',
    },
    {
        'class': 4,
        'Description': 'kesalahan kata jamak',
        'salah': 'mereka-mereka',
        'betul': 'mereka',
    },
    {
        'class': 5,
        'Description': 'kesalahan kata penguat',
        'salah': 'sangat tinggi sekali',
        'betul': 'sangat tinggi',
    },
    {
        'class': 6,
        'Description': 'kata adjektif dan imbuhan "ter" tanpa penguat.',
        'salah': 'Sani mendapat markah yang tertinggi sekali.',
        'betul': 'Sani mendapat markah yang tertinggi.',
    },
    {
        'class': 7,
        'Description': 'kesalahan kata hubung',
        'salah': 'Sally sedang membaca bila saya tiba di rumahnya.',
        'betul': 'Sally sedang membaca apabila saya tiba di rumahnya.',
    },
    {
        'class': 8,
        'Description': 'kesalahan kata bilangan',
        'salah': 'Beribu peniaga tidak membayar cukai pendapatan.',
        'betul': 'Beribu-ribu peniaga tidak membayar cukai pendapatan',
    },
    {
        'class': 9,
        'Description': 'kesalahan kata sendi',
        'salah': 'Umar telah berpindah daripada sekolah ini bulan lalu.',
        'betul': 'Umar telah berpindah dari sekolah ini bulan lalu.',
    },
    {
        'class': 10,
        'Description': 'kesalahan penjodoh bilangan',
        'salah': 'Setiap orang pelajar',
        'betul': 'Setiap pelajar.',
    },
    {
        'class': 11,
        'Description': 'kesalahan kata ganti diri',
        'salah': 'Pencuri itu telah ditangkap. Beliau dibawa ke balai polis.',
        'betul': 'Pencuri itu telah ditangkap. Dia dibawa ke balai polis.',
    },
    {
        'class': 12,
        'Description': 'kesalahan ayat pasif',
        'salah': 'Cerpen itu telah dikarang oleh saya.',
        'betul': 'Cerpen itu telah saya karang.',
    },
    {
        'class': 13,
        'Description': 'kesalahan kata tanya',
        'salah': 'Kamu berasal dari manakah ?',
        'betul': 'Kamu berasal dari mana ?',
    },
    {
        'class': 14,
        'Description': 'kesalahan tanda baca',
        'salah': 'Kamu berasal dari manakah .',
        'betul': 'Kamu berasal dari mana ?',
    },
    {
        'class': 15,
        'Description': 'kesalahan kata kerja tak transitif',
        'salah': 'Dia kata kepada saya',
        'betul': 'Dia berkata kepada saya',
    },
    {
        'class': 16,
        'Description': 'kesalahan kata kerja tak transitif',
        'salah': 'Dia kata kepada saya',
        'betul': 'Dia berkata kepada saya',
    },
    {
        'class': 17,
        'Description': 'kesalahan kata kerja transitif',
        'salah': 'Dia suka baca buku',
        'betul': 'Dia suka membaca buku',
    },
    {
        'class': 18,
        'Description': 'penggunaan kata yang tidak tepat',
        'salah': 'Tembuk Besar negeri Cina dibina oleh Shih Huang Ti.',
        'betul': 'Tembok Besar negeri Cina dibina oleh Shih Huang Ti',
    },
    {
        'class': 19,
        'Description': 'kesalahan frasa kerja tak transitif',
        'salah': 'berdasarkan pada keterangan ini',
        'betul': 'berdasarkan keterangan ini',
    },
    {
        'class': 20,
        'Description': 'kesalahan frasa kerja transitif',
        'salah': 'Dia membeli banyak buah',
        'betul': 'Dia banyak membeli buah',
    },
    {
        'class': 21,
        'Description': 'kesalahan frasa kerja pasif',
        'salah': 'Surat itu saga akan balas',
        'betul': 'Surat itu akan saga balas',
    },
]


class Tatabahasa:
    def __init__(self, d):
        self.d = d
        self.kesalahan = {i['Description']: no for no, i in enumerate(self.d)}
        self.reverse_kesalahan = {v: k for k, v in self.kesalahan.items()}
        self.vocab_size = len(self.d)

    def encode(self, s):
        return [self.kesalahan[i] for i in s]

    def decode(self, ids, strip_extraneous = False):
        return [self.reverse_kesalahan[i] for i in ids]

In [5]:
@registry.register_problem
class Grammar(text_problems.Text2TextProblem):
    """grammatical error correction."""

    def feature_encoders(self, data_dir):
        encoder = Encoder(sp)
        t = Tatabahasa(d)
        return {'inputs': encoder, 'targets': encoder, 'targets_error_tag': t}

    def hparams(self, defaults, model_hparams):
        super(Grammar, self).hparams(defaults, model_hparams)
        if 'use_error_tags' not in model_hparams:
            model_hparams.add_hparam('use_error_tags', True)
        if 'middle_prediction' not in model_hparams:
            model_hparams.add_hparam('middle_prediction', False)
        if 'middle_prediction_layer_factor' not in model_hparams:
            model_hparams.add_hparam('middle_prediction_layer_factor', 2)
        if 'ffn_in_prediction_cascade' not in model_hparams:
            model_hparams.add_hparam('ffn_in_prediction_cascade', 1)
        if 'error_tag_embed_size' not in model_hparams:
            model_hparams.add_hparam('error_tag_embed_size', 12)
        if model_hparams.use_error_tags:
            defaults.modality[
                'targets_error_tag'
            ] = modalities.ModalityType.SYMBOL
            error_tag_vocab_size = self._encoders[
                'targets_error_tag'
            ].vocab_size
            defaults.vocab_size['targets_error_tag'] = error_tag_vocab_size

    def example_reading_spec(self):
        data_fields, _ = super(Seq2edits, self).example_reading_spec()
        data_fields['targets_error_tag'] = tf.VarLenFeature(tf.int64)
        return data_fields, None

    @property
    def approx_vocab_size(self):
        return 32100

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 200},
            {'split': problem.DatasetSplit.EVAL, 'shards': 1},
        ]

In [6]:
os.system('mkdir t2t-tatabahasa/train-small')
DATA_DIR = os.path.expanduser('t2t-tatabahasa/data')
TMP_DIR = os.path.expanduser('t2t-tatabahasa/tmp')
TRAIN_DIR = os.path.expanduser('t2t-tatabahasa/train-small')

In [7]:
PROBLEM = 'grammar'
t2t_problem = problems.problem(PROBLEM)

In [8]:
MODEL = 'transformer_tag'
HPARAMS = 'transformer_base'

In [9]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib

In [10]:
X = tf.placeholder(tf.int32, [None, None], name = 'x_placeholder')
Y = tf.placeholder(tf.int32, [None, None], name = 'y_placeholder')
targets_error_tag = tf.placeholder(tf.int32, [None, None], 'error_placeholder')
X_seq_len = tf.count_nonzero(X, 1, dtype=tf.int32)
maxlen_decode = tf.reduce_max(X_seq_len)

x = tf.expand_dims(tf.expand_dims(X, -1), -1)
y = tf.expand_dims(tf.expand_dims(Y, -1), -1)
targets_error_tag_ = tf.expand_dims(tf.expand_dims(targets_error_tag, -1), -1)

features = {
    "inputs": x,
    "targets": y,
    "target_space_id": tf.constant(1, dtype=tf.int32),
    'targets_error_tag': targets_error_tag,
}
Modes = tf.estimator.ModeKeys
hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


In [11]:
hparams.filter_size = 2048
hparams.hidden_size = 512
hparams.num_heads = 8
hparams.num_hidden_layers = 6
hparams.num_decoder_layers = hparams.num_hidden_layers
hparams.num_encoder_layers = hparams.num_hidden_layers 
hparams.vocab_divisor = 128
hparams.label_smoothing = 0.0
hparams.shared_embedding_and_softmax_weights = False
hparams.dropout = 0.1
hparams.max_length = 1024
hparams.multiproblem_mixing_schedule = "pretrain"

hparams.optimizer = "Adafactor"
hparams.learning_rate_warmup_steps = 10000
hparams.learning_rate_schedule = "rsqrt_decay"

In [12]:
model = registry.model(MODEL)(hparams, Modes.PREDICT)

INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.label_smoothing to 0.0


...

INFO:tensorflow:Setting hparams.symbol_dropout to 0.0


INFO:tensorflow:Setting hparams.attention_dropout to 0.0


INFO:tensorflow:Setting hparams.attention_dropout to 0.0


INFO:tensorflow:Setting hparams.relu_dropout to 0.0


INFO:tensorflow:Setting hparams.relu_dropout to 0.0


In [13]:
# logits = model(features)
# logits

# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())
# l = sess.run(logits, feed_dict = {X: [[10,10, 10, 10,10,1],[10,10, 10, 10,10,1]],
#                              Y: [[10,10, 10, 10,10,1],[10,10, 10, 10,10,1]],
#                              targets_error_tag: [[10,10, 10, 10,10,1],
#                                                 [10,10, 10, 10,10,1]]})

In [14]:
features = {
    "inputs": x,
    "target_space_id": tf.constant(1, dtype=tf.int32),
}

with tf.variable_scope(tf.get_variable_scope(), reuse = False):
    fast_result = model._greedy_infer(features, maxlen_decode)

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


In [15]:
ckpt_path = tf.train.latest_checkpoint(os.path.join(TRAIN_DIR))
ckpt_path

't2t-tatabahasa/train-small/model.ckpt-200'

In [16]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [17]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from t2t-tatabahasa/train-small/model.ckpt-200


INFO:tensorflow:Restoring parameters from t2t-tatabahasa/train-small/model.ckpt-200


In [18]:
import pickle

with open('../pure-text/dataset-tatabahasa.pkl', 'rb') as fopen:
    data = pickle.load(fopen)

encoder = Encoder(sp)

In [27]:
def get_xy(row, encoder):
    x, y, tag = [], [], []

    for i in range(len(row[0])):
        t = encoder.encode(row[0][i][0])
        y.extend(t)
        t = encoder.encode(row[1][i][0])
        x.extend(t)
        tag.extend([row[1][i][1]] * len(t))

    # EOS
    x.append(1)
    y.append(1)
    tag.append(0)

    return x, y, tag

In [36]:
import numpy as np

In [28]:
x, y, tag = get_xy(data[0], encoder)

In [29]:
r = sess.run(fast_result, 
         feed_dict = {X: [x]})

In [42]:
r['outputs_tag']

array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 0]])

In [40]:
np.array(tag)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 0])

In [31]:
r['outputs']

array([[  104,  6892,  3208,    13,    13,    25,     7,   749,    36,
           15,     6, 15277,   844,    13,   564, 15277,   844,    13,
          564,    15,     4,  2083,   417,   727,  4073,    15,     5,
           34,   394,   648,   714,  1337,    17,   798,    18,  3481,
         4963,    15,     3,     1]])

In [32]:
encoder.decode(r['outputs'][0].tolist())

'Dirk Janaas-Jan " Huntelaar Huntelaar ( lahir 12 Ogos 1983 ) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang .'

In [33]:
encoder.decode(x)

'Dirk Jan Klaas " Klaas-Jan " Huntelaar ( lahir 12 Ogos 1983 ) merupakan bola sepak Belanda pemain yang bermain di penyerang posisi .'